In [56]:
import spacy
import nltk
from pprint import pprint
import random
from sklearn import preprocessing
nltk.download("all")

nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/miles/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /home/miles/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]   

In [57]:
import os
def load_corpus(folder):
    corpus = []
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            try:
                with open(os.path.join(root, name), "r") as fp:
                    corpus.append(fp.read())
            except UnicodeDecodeError as e:
                print(e.__str__(), "for", os.path.join(root, name))
    return corpus

def load_corpuses(folder):
    sub_folders = []
    for root, dirs, files in os.walk(folder):
        if dirs:
            for dir_ in dirs:
                sub_folders.append(dir_)

    corpuses = {}
    print(sub_folders)
    for sub_folder in sub_folders:
        print(os.path.join(folder, sub_folder))
        corpuses[sub_folder] = load_corpus(os.path.join(folder, sub_folder))
    return corpuses

corpuses = load_corpuses("bbc")

['business', 'tech', 'entertainment', 'sport', 'politics']
bbc/business
bbc/tech
bbc/entertainment
bbc/sport
'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte for bbc/sport/199.txt
bbc/politics


In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(stop_words=stopwords.words('english'))

all_ = []
for corpus in corpuses:
    all_ += corpuses[corpus]

vectorizer.fit(all_)
# x = vectorizer.transform(corpuses["tech"])
# vectorizer.vocabulary_.get('Sony')

CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [59]:
vectorizer.transform(['regulatory approval']).toarray()[0]


array([0, 0, 0, ..., 0, 0, 0])

In [60]:
import math
x = []
y =[]
from tqdm.notebook import tqdm

entity_types = preprocessing.LabelEncoder()
entity_types.fit(['CARDINAL', 'PERSON', 'GPE', 'MONEY', 'ORG', 'ORDINAL', 'WORK_OF_ART', 'NORP', 'PERCENT', 'DATE', 'LANGUAGE', 'FAC', 'LOC', 'TIME', 'PRODUCT', 'EVENT', 'QUANTITY', 'LAW'])
for corpus in tqdm(corpuses):
    for story in tqdm(corpuses[corpus]):
        analysed = nlp(story)
        x.append(vectorizer.transform([story]).toarray()[0] + entity_types.transform([tag.label_ for tag in analysed.ents])[0])
        y.append(corpus)
pprint(x[:3])
pprint(y[:3])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/510 [00:00<?, ?it/s]

  0%|          | 0/401 [00:00<?, ?it/s]

  0%|          | 0/386 [00:00<?, ?it/s]

  0%|          | 0/510 [00:00<?, ?it/s]

  0%|          | 0/417 [00:00<?, ?it/s]

[array([13, 13, 13, ..., 13, 13, 13]),
 array([17, 17, 17, ..., 17, 17, 17]),
 array([11, 11, 11, ..., 11, 11, 11])]
['business', 'business', 'business']


In [61]:
size_dataset_full=len(x)
size_test=int(round(size_dataset_full*0.2,0))

list_test_indices=random.sample(range(size_dataset_full), size_test)

test_x = []
test_y = []
train_x = []
train_y = []

for i,example in enumerate(x):
  if i in list_test_indices:
      test_x.append(example)
      test_y.append(y[i])
  else:
      train_x.append(example)
      train_y.append(y[i])

le = preprocessing.LabelEncoder()
le.fit(train_y + test_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)

In [62]:
print(test_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2]


In [63]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

svm_clf=make_pipeline(StandardScaler(), svm.SVC(cache_size=2000, decision_function_shape='ovo'))

In [64]:
svm_clf.fit(train_x, train_y)
# train_x[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(cache_size=2000, decision_function_shape='ovo'))])

In [65]:
Y_text_predictions = svm_clf.predict(test_x)

In [66]:
from sklearn.metrics import classification_report
print(classification_report(test_y, Y_text_predictions))

              precision    recall  f1-score   support

           0       0.31      0.85      0.45       100
           1       0.00      0.00      0.00        87
           2       0.00      0.00      0.00        82
           3       0.34      0.39      0.37       104
           4       0.33      0.21      0.25        72

    accuracy                           0.32       445
   macro avg       0.19      0.29      0.21       445
weighted avg       0.20      0.32      0.23       445

/home/miles/.local/share/virtualenvs/CMT316-_3fUo8fQ/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/miles/.local/share/virtualenvs/CMT316-_3fUo8fQ/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision an

In [67]:
story = """
Isohnson is expected to unveil a traffic light system for foreign travel in an update on measures for easing Covid restrictions in England.

The prime minister is to make a series of announcements on changes to lockdown rules at a Downing Street briefing.

More detail might be given on the possible use of Covid certificates to access services or events domestically. 
"""

le.inverse_transform([svm_clf.predict([vectorizer.transform([story]).toarray()[0]])[0]])[0]

'tech'